# 📘 Progress Callbacks for Optimization Monitoring> Monitor optimization progress with progress bars, logging, and early stopping⏱️ **15-20 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Feature Demo**---

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Use `ProgressBar` for real-time optimization monitoring2. Log optimization history with `IterationLogger`3. Prevent wasted iterations with `EarlyStopping`4. Combine multiple callbacks using `CallbackChain`5. Create custom callbacks for specialized needs---

## 🔬 Feature Overview**What problem does this solve?**- Long-running fits can appear frozen without feedback- Debugging requires iteration-level details- Optimization may continue unnecessarily after convergence**Available callbacks:**- `ProgressBar`: tqdm-based progress tracking- `IterationLogger`: Detailed logging to file- `EarlyStopping`: Stop when no improvement detected- `CallbackChain`: Combine multiple callbacks- `CallbackBase`: Base class for custom callbacks---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit
from nlsq.callbacks import (
    CallbackBase,
    CallbackChain,
    EarlyStopping,
    IterationLogger,
    ProgressBar,
)


def exponential_decay(x, amplitude, rate, offset):
    return amplitude * jnp.exp(-rate * x) + offset

# Generate sample data
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 100 * np.exp(-0.5 * x) + 10
y = y_true + np.random.normal(0, 3, size=len(x))

## Example 1: Simple Progress BarMonitor optimization progress with a visual progress bar.

In [ ]:
# Create progress bar callback
callback = ProgressBar(max_nfev=50, desc="Fitting exponential")

# Fit with progress bar
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print(f'✓ Fitted: amplitude={popt[0]:.2f}, rate={popt[1]:.3f}, offset={popt[2]:.2f}')

## Example 2: Iteration LoggingLog detailed optimization progress to a file for later analysis.

In [ ]:
# Create logging callback
callback = IterationLogger(
    filename='optimization.log',
    mode='w',
    log_params=True  # Include parameter values
)

# Fit with logging
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print('✓ Log written to optimization.log')
print('First few lines:')
with open('optimization.log') as f:
    print(''.join(f.readlines()[:10]))

## Example 3: Early StoppingPrevent wasted iterations by stopping when optimization stalls.

In [ ]:
# Create early stopping callback
callback = EarlyStopping(
    patience=10,       # Stop after 10 iterations without improvement
    min_delta=1e-6,    # Minimum improvement threshold
    verbose=True)

# Fit with early stopping
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=1000  # Set high, early stopping prevents waste
)

print(f'✓ Fitted: amplitude={popt[0]:.2f}, rate={popt[1]:.3f}, offset={popt[2]:.2f}')

## Example 4: Combining Multiple CallbacksUse `CallbackChain` to combine progress bar, logging, and early stopping.

In [ ]:
# Combine multiple callbacks
callback = CallbackChain(
    ProgressBar(max_nfev=50, desc="Optimizing"),
    IterationLogger('combined.log', log_params=False),
    EarlyStopping(patience=10, verbose=False))

# Fit with callback chain
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print('✓ All callbacks executed!')
print('Check combined.log for iteration history')

## Example 5: Custom CallbackCreate specialized callbacks by subclassing `CallbackBase`.

In [ ]:
class BestParameterTracker(CallbackBase):
    """Custom callback to track best parameters."""

    def __init__(self):
        self.best_cost = np.inf
        self.best_params = None
        self.history = []

    def __call__(self, iteration, cost, params, info):
        """Track best parameters."""
        self.history.append({'iter': iteration, 'cost': cost})

        if cost < self.best_cost:
            self.best_cost = cost
            self.best_params = params.copy()
            print(f'  → New best at iter {iteration}: cost={cost:.6f}')

    def get_best(self):
        return self.best_params, self.best_cost

# Use custom callback
tracker = BestParameterTracker()
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=tracker,
    max_nfev=50)

best_params, best_cost = tracker.get_best()
print(f'\n✓ Best cost: {best_cost:.6f}')
print(f'✓ Final params match best: {np.allclose(popt, best_params)}')

## Visualize Results

In [ ]:
y_fit = exponential_decay(x, *popt)

fig = plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.plot(x, y, 'o', alpha=0.3, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, y_fit, 'r-', linewidth=2, label='Fitted')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Exponential Decay Fit')

plt.subplot(122)
plt.plot(x, y - y_fit, '.')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('x')
plt.ylabel('Residuals')
plt.title('Fit Quality')

plt.tight_layout()
plt.tight_layout()
display(fig)
plt.close(fig)


## 💡 Key Insights1. **Progress bars** provide real-time feedback for long-running fits2. **Iteration logging** enables detailed post-analysis and debugging3. **Early stopping** saves computation when optimization stalls4. **Callback chains** combine multiple monitoring strategies5. **Custom callbacks** enable specialized monitoring for your use case---## 📚 Best Practices- Use `ProgressBar` for interactive work (notebooks, scripts)- Use `IterationLogger` for production/automated workflows- Set `patience` in `EarlyStopping` based on problem complexity- Combine callbacks with `CallbackChain` for comprehensive monitoring- Extend `CallbackBase` for custom visualization or metrics---## 🎓 Next Steps- Try callbacks on your own optimization problems- Create custom callbacks for domain-specific metrics- Explore callback integration with logging frameworks- Use callbacks to implement adaptive optimization strategies---